# This will be the model traning part

In [63]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display='diagram')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [64]:
dataframe=pd.read_csv('data/StudentsPerformance.csv')

In [65]:
dataframe.head(5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [66]:
dataframe['total_marks']=dataframe['math score']+dataframe['reading score']+dataframe['writing score']

In [67]:
def check_marks(x):
    avergae_marks=np.median(dataframe['total_marks'])
    if x>avergae_marks:
        return "Good"
    else:
        return "Bad"

In [68]:
dataframe['result']=dataframe['total_marks'].apply(check_marks)

In [69]:
dataframe

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total_marks,result
0,female,group B,bachelor's degree,standard,none,72,72,74,218,Good
1,female,group C,some college,standard,completed,69,90,88,247,Good
2,female,group B,master's degree,standard,none,90,95,93,278,Good
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,Bad
4,male,group C,some college,standard,none,76,78,75,229,Good
...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,282,Good
996,male,group C,high school,free/reduced,none,62,55,55,172,Bad
997,female,group C,high school,free/reduced,completed,59,71,65,195,Bad
998,female,group D,some college,standard,completed,68,78,77,223,Good


In [70]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   gender                       1000 non-null   object
 1   race/ethnicity               1000 non-null   object
 2   parental level of education  1000 non-null   object
 3   lunch                        1000 non-null   object
 4   test preparation course      1000 non-null   object
 5   math score                   1000 non-null   int64 
 6   reading score                1000 non-null   int64 
 7   writing score                1000 non-null   int64 
 8   total_marks                  1000 non-null   int64 
 9   result                       1000 non-null   object
dtypes: int64(4), object(6)
memory usage: 78.2+ KB


In [71]:
numerical_data=[col for col in dataframe.columns if dataframe[col].dtype!='O']
categorical_data=[col for col in dataframe.columns if dataframe[col].dtype=='O']

In [72]:
print('Numerical Data',numerical_data)
print('Categroical Data',categorical_data)

Numerical Data ['math score', 'reading score', 'writing score', 'total_marks']
Categroical Data ['gender', 'race/ethnicity', 'parental level of education', 'lunch', 'test preparation course', 'result']


In [73]:
pipe=Pipeline([
    ('OHE',OneHotEncoder(sparse=False)),
    ('Scaler',StandardScaler()),
    ('Model',LogisticRegression())
])

In [74]:
pipe

Pipeline(steps=[('OHE', OneHotEncoder(sparse=False)),
                ('Scaler', StandardScaler()), ('Model', LogisticRegression())])

In [75]:
X = dataframe.drop('result', axis=1)
y = dataframe['result']

numerical_cols = X.select_dtypes(exclude='object').columns
categorical_cols = X.select_dtypes(include='object').columns


In [76]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

model_pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression())
    ]
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)

model_pipeline.fit(X_train, y_train)

y_pred = model_pipeline.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.9966666666666667
              precision    recall  f1-score   support

         Bad       1.00      0.99      1.00       151
        Good       0.99      1.00      1.00       149

    accuracy                           1.00       300
   macro avg       1.00      1.00      1.00       300
weighted avg       1.00      1.00      1.00       300

